# Importe de librerias

In [ ]:
import pandas as pd
import lightning as L
from lightning.pytorch.callbacks import ModelCheckpoint
import os
from sklearn.model_selection import train_test_split
import torch

torch.set_float32_matmul_precision('high')

In [2]:
from dataset import LitPriceData
from trainer import LitTrainer

In [3]:
from utils import splitData

# Carga de monedas selecionadas

In [4]:
coins = pd.read_json('../data/selected_coins.json')
coins

,0
0,celestia
1,flare-networks
2,polygon-bridged-usdt-polygon
3,xdce-crowd-sale
4,injective-protocol
5,virtual-protocol
6,blockstack
7,sonic-3
8,binance-bridged-usdc-bnb-smart-chain
9,mantle-staked-ether


In [5]:
def trainModel(model, X, y, checkpoint_callback=None):
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
    dataModule = LitPriceData(X_train, y_train, X_val, y_val)
       
    if checkpoint_callback is not None:
        trainer = L.Trainer(max_epochs=75, accelerator="auto", callbacks=[checkpoint_callback])
    else:
        trainer = L.Trainer(max_epochs=75, accelerator="auto")

    trainer.fit(model=model, datamodule=dataModule)

In [6]:
hyperparams = {'hidden_size': 76,
 'num_layers': 2,
 'lr': 0.005094100053762949,
 'dropout': 0.4197932446614159,
 'sequence_legth': 12}

for i, row in coins.iterrows():    
    name = row[0]
    export_dir = f"../models/exports/{name}"
    os.makedirs(export_dir, exist_ok=True)

    checkpoint_callback = ModelCheckpoint(
        dirpath = f"../models/checkpoints/{name}",
        filename = "checkpoints",
        save_top_k=1,
        monitor="val_loss",
        mode="min"
    )

    #Change the values
    model = LitTrainer(hidden_size=hyperparams["hidden_size"], num_layers=hyperparams["num_layers"], lr=hyperparams["lr"], dropout=hyperparams["dropout"])
    X, y = splitData(name, sequence_length=hyperparams["sequence_legth"])
    trainModel(model, X, y, checkpoint_callback)
    
    best_ckpt_path = checkpoint_callback.best_model_path
    print(f"Best checkpoint for {name}: {best_ckpt_path}")

    best_model = LitTrainer.load_from_checkpoint(best_ckpt_path, hidden_size=hyperparams["hidden_size"], num_layers=hyperparams["num_layers"], dropout=hyperparams["dropout"])

    export_state_dict_path = os.path.join(export_dir, "best_model.pth")
    torch.save(best_model.state_dict(), export_state_dict_path)


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/home/giorgio6846/miniconda3/envs/TF-ML/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/logger_connector/logger_connector.py:76: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `lightning.pytorch` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
/home/giorgio6846/miniconda3/envs/TF-ML/lib/python3.11/site-packages/lightning/pytorch/callbacks/model_checkpoint.py:658: Checkpoint directory /home/giorgio6846/Code/Clases/Machine Learning/TF/models/checkpoints/celestia exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type      | Params | Mode 
---

Sanity Checking DataLoader 0:   0%|          | 0/2 [00:00<?, ?it/s]

/home/giorgio6846/miniconda3/envs/TF-ML/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.
/home/giorgio6846/Code/Clases/Machine Learning/TF/src/trainer.py:23: UserWarning: Using a target size (torch.Size([64, 7])) that is different to the input size (torch.Size([64, 5])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  loss = F.mse_loss(out, y)


RuntimeError: The size of tensor a (5) must match the size of tensor b (7) at non-singleton dimension 1